In [ ]:
!pip install numpy-stl
import numpy as np
import pandas as pd
from stl import mesh

# Config section
CSV_PATH = "R134a_PVT_data.csv"       # path to input data file
OUT_STL = "R134a_PvT_Surface.stl"     # output STL file name

# Choose which logarithm to use
log_fn = np.log      # use np.log10 if you want base-10 logarithm

# Load and clean the data
df = pd.read_csv(CSV_PATH)
df = df.dropna(subset=['Pressure_Pa', 'Temperature_K', 'SpecificVolume_m3_per_kg'])
if df.empty:
    raise SystemExit("No valid data after dropping NaNs.")

# Compute transformed axes
df['log_v'] = log_fn(df['SpecificVolume_m3_per_kg'].astype(float))
df['log_p'] = log_fn(df['Pressure_Pa'].astype(float))
df['T'] = df['Temperature_K'].astype(float)

# Round for stable grouping
df['log_v_r'] = df['log_v'].round(12)
df['T_r'] = df['T'].round(9)

# Build grid from unique values (expected for evenly sampled surface)
unique_v = np.sort(df['log_v_r'].unique())
unique_T = np.sort(df['T_r'].unique())
n_v = unique_v.size
n_T = unique_T.size

# If the data forms a complete grid, use it directly; otherwise, use nearest-neighbor binning
if n_v * n_T == len(df):
    mapping = df.set_index(['T_r', 'log_v_r'])['log_p'].to_dict()
    Z = np.full((n_T, n_v), np.nan, dtype=float)
    for i, Tval in enumerate(unique_T):
        for j, vval in enumerate(unique_v):
            Z[i, j] = mapping.get((Tval, vval), np.nan)
else:
    print("Warning: data not a perfect grid — performing nearest-neighbor binning.")
    n = int(np.sqrt(len(df)))
    n_T = n_v = n if n > 1 else max(n_T, n_v, 2)
    grid_v = np.linspace(df['log_v'].min(), df['log_v'].max(), n_v)
    grid_T = np.linspace(df['T'].min(), df['T'].max(), n_T)
    Z = np.full((n_T, n_v), np.nan, dtype=float)
    idx_v = np.searchsorted(grid_v, df['log_v'].values, side='left')
    idx_T = np.searchsorted(grid_T, df['T'].values, side='left')
    idx_v = np.clip(idx_v, 0, n_v-1)
    idx_T = np.clip(idx_T, 0, n_T-1)
    count = np.zeros_like(Z)
    sumZ = np.zeros_like(Z)
    for k, (iv, it) in enumerate(zip(idx_v, idx_T)):
        sumZ[it, iv] += df['log_p'].iat[k]
        count[it, iv] += 1
    mask = count > 0
    Z[mask] = sumZ[mask] / count[mask]

# Scale the axes (X, Y, Z) to a 0–100 range for easier 3D printing
X_raw = unique_v if (n_v * n_T == len(df)) else np.linspace(df['log_v'].min(), df['log_v'].max(), n_v)
Y_raw = unique_T if (n_v * n_T == len(df)) else np.linspace(df['T'].min(), df['T'].max(), n_T)
Z_raw = Z

nan_ratio = np.isnan(Z_raw).sum() / Z_raw.size
if nan_ratio > 0.3:
    print(f"Warning: {nan_ratio*100:.1f}% of grid cells are empty; STL will skip those areas.")

def scale_to_0_100(arr):
    arr = np.asarray(arr, dtype=float)
    mn, mx = np.nanmin(arr), np.nanmax(arr)
    if np.isclose(mn, mx):
        return np.zeros_like(arr, dtype=float) + 50.0
    return (arr - mn) / (mx - mn) * 100.0

X = scale_to_0_100(X_raw)
Y = scale_to_0_100(Y_raw)
Zs = scale_to_0_100(Z_raw)

# Build triangular faces (two triangles per grid cell)
verts = []
faces = []
vertex_indices = -np.ones((n_T, n_v), dtype=int)

for i in range(n_T):
    for j in range(n_v):
        if np.isnan(Zs[i, j]):
            continue
        vtx = [float(X[j]), float(Y[i]), float(Zs[i, j])]
        vertex_indices[i, j] = len(verts)
        verts.append(vtx)

for i in range(n_T - 1):
    for j in range(n_v - 1):
        a = vertex_indices[i, j]
        b = vertex_indices[i+1, j]
        c = vertex_indices[i, j+1]
        d = vertex_indices[i+1, j+1]
        if a < 0 or b < 0 or c < 0 or d < 0:
            continue
        faces.append([a, b, c])
        faces.append([b, d, c])

if len(faces) == 0:
    raise SystemExit("No faces generated: check input data and gridding.")

# Build numpy arrays for STL output
verts_np = np.array(verts, dtype=float)
faces_np = np.array(faces, dtype=int)

# Create the STL mesh
model = mesh.Mesh(np.zeros(faces_np.shape[0], dtype=mesh.Mesh.dtype))
for i_face, (ia, ib, ic) in enumerate(faces_np):
    model.vectors[i_face][0] = verts_np[ia]
    model.vectors[i_face][1] = verts_np[ib]
    model.vectors[i_face][2] = verts_np[ic]

# Save the STL file
model.save(OUT_STL)
print(f"STL saved to: {OUT_STL}")
print(f"Grid: n_v={n_v}, n_T={n_T}, vertices={len(verts)}, faces={len(faces)}")


FileNotFoundError: [Errno 2] No such file or directory: 'R134a_PVT_data.csv'